In [1]:
# !pip install flask

In [89]:
import os
import json
import time
import requests
from flask import Flask, Response
import functools
from datetime import datetime, timedelta

import pymysql
import numpy as np
import pandas as pd

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

from model import create_model

from multiprocessing import cpu_count, Pool

tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

from dotenv import load_dotenv
load_dotenv()

True

In [90]:
app = Flask(__name__)
API_KEY = os.getenv('API_KEY')

In [91]:
# DB_CONN INFOS
DB_USER = os.getenv('MYSQL_USER')
DB_PASSWD = os.getenv('MYSQL_PASSWORD')
DB_HOST = os.getenv('MYSQL_HOST')
DB_DB = os.getenv('MYSQL_DATABASE')

# Connect to db
db = pymysql.connect(
    user=DB_USER, 
    passwd=DB_PASSWD, 
    host=DB_HOST, 
    db=DB_DB, 
    charset='utf8'
)

# Set cursor
cursor = db.cursor(pymysql.cursors.DictCursor)

In [92]:
def requestCurrentInfo(startIdx, endIdx):
    url = "http://openapi.seoul.go.kr:8088/{}/json/bikeList/{}/{}".format(API_KEY, startIdx, endIdx)
    res = requests.get(url)
    res = res.json()
    
    if res['rentBikeStatus']['RESULT']['CODE'] != 'INFO-000':
        raise Exception('Failed to get data from api.')
    
    return res['rentBikeStatus']['row']

def getAllCombinedInfo():
    res = []
    for i in range(1, 1001 + 1, 1000):
        startIdx = i
        endIdx = i + 999
        res = res + requestCurrentInfo(startIdx, endIdx)
        
    return res

In [93]:
def cache(seconds: int, maxsize: int = 2050, typed: bool = False):
    def wrapper_cache(func):
        func = functools.lru_cache(maxsize=maxsize, typed=typed)(func)
        func.delta = timedelta(seconds=seconds)
        func.expiration = datetime.utcnow() + func.delta

        @functools.wraps(func)
        def wrapped_func(*args, **kwargs):
            if datetime.utcnow() >= func.expiration:
                func.cache_clear()
                func.expiration = datetime.utcnow() + func.delta

            return func(*args, **kwargs)

        return wrapped_func

    return wrapper_cache

In [94]:
def get_last_10rows(stationId):
    sql = """
    SELECT parkingBikeTotCnt FROM (
        SELECT * FROM `{}` ORDER BY idx DESC LIMIT 10
    ) sub
    ORDER BY idx ASC
    """.format(stationId)

    cursor.execute(sql)
    res = cursor.fetchall()
    res = [row['parkingBikeTotCnt'] for row in res]
    return np.array(res)

In [95]:
DAY_IN_SEC = 60 * 60 * 24
@cache(seconds=DAY_IN_SEC)
def get_model(stationId):
    path = "./models/{}.h5".format(stationId)
    model = keras.models.load_model(path)
    return model


def forecastFutureOfStation(station):
    stationId = station['stationId']
    model = get_model(stationId)
    x = get_last_10rows(stationId)
    
    scaler = MinMaxScaler()
    x = scaler.fit_transform(pd.DataFrame(x))
    x = x.reshape(1, x.shape[0], 1)
    
    x = tf.keras.backend.constant(x)
    y = model(x).numpy()
    
    y = y.reshape(y.shape[0], y.shape[1])
    station['future'] = scaler.inverse_transform(y)
    return station

In [96]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
MODELS_PATH = "./models/"

@app.route("/stations/available")
@cache(seconds=2*60)
def get_stations():
    def isAvailableStation(station):
        if station['stationId'] in availableStations:
            return station
        else:
            return None
        
    file_list = os.listdir(MODELS_PATH)
    
    availableStations = list(map(lambda x: x.split('.')[0], file_list))
    allStations = getAllCombinedInfo()
    availableStations = list(filter(isAvailableStation, allStations))
    
    nCores = cpu_count()
    with Pool(nCores, maxtasksperchild=8) as pool:
        result = pool.map(forecastFutureOfStation, availableStations)
        
    resultInJson = json.dumps(result, ensure_ascii=False, cls=NumpyEncoder)
    return Response(resultInJson, mimetype='application/json')

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
1.231.99.37 - - [02/Aug/2020 17:03:55] "GET /stations/available HTTP/1.1" 200 -
